#1
Ошибка 500 возникает в результате исполнения кода на backend и зачастую связана
с проблемами в самом коде. Поэтому в случае ее возникновения необходимо
тщательно проанализировать код backend, его логику, в идеале - посмотреть на 
результаты выполнения автоматических тестов, все ли проходит успешно.
Также такая ошибка может возникнуть из-за неправильной конфигурации сервера, это
решается чтением документации и проверкой параметров работы сервера.

#2

In [1]:
def create_handlers(callback):
  handlers = []
  for step in range(5):
    handlers.append(lambda: callback(step))
  return handlers

def execute_handlers(handlers):
  for handler in handlers:
    handler()

def callback(step):
  print(f"callback called with argument: {step}")

handlers = create_handlers(callback)
execute_handlers(handlers)

callback called with argument: 4
callback called with argument: 4
callback called with argument: 4
callback called with argument: 4
callback called with argument: 4


Отладка кода показывает, что все handler обладают одним и тем же аргументом - 
4.
Это связано с особенностью языка Python, а именно поздним связыванием
(late binding) внутри замыкания (closure). Значение step не прикрепляется
к handler до тех пор, пока не будет вызван сам handler.
Решением проблемы является прикрепление step к handler через
functools.

In [2]:
import functools
from typing import Callable, List

def create_handlers(callback: Callable[[int], None]) -> List[Callable[[], None]]:
  handlers = []
  for step in range(5):
    handlers.append(functools.partial(callback, step))
  return handlers

def execute_handlers(handlers: List[Callable[[], None]]):
  for handler in handlers:
    handler()

def callback(step: int):
  print(f"callback called with argument: {step}")

handlers = create_handlers(callback)
execute_handlers(handlers)

callback called with argument: 0
callback called with argument: 1
callback called with argument: 2
callback called with argument: 3
callback called with argument: 4


#3

In [3]:
from collections import defaultdict
from bs4 import BeautifulSoup
from typing import Dict
import requests

def get_tag_counts(soup) -> Dict[str, int]:
    tag_counts = defaultdict(int)
    for tag in soup.find_all():
        tag_counts[tag.name] += 1
    return dict(tag_counts)

def get_tag_counts_with_attributes(soup) -> Dict[str, int]:
    tag_counts = defaultdict(int)
    for tag in soup.find_all():
        if tag.attrs:
            tag_counts[tag.name] += 1
    return dict(tag_counts)

def count_tags(url: str) -> Dict[str, int]:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    tag_counts = get_tag_counts(soup)
    tag_counts_with_attributes = get_tag_counts_with_attributes(soup)
    return {
        "total_tag_count": sum(tag_counts.values()),
        "total_tag_with_attr_count": sum(tag_counts_with_attributes.values()),
    }

url = "https://greenatom.ru"
result = count_tags(url)

print(f"Total number of HTML tags: {result['total_tag_count']}\n")
print(f"Total number of HTML tags with attributes: {result['total_tag_with_attr_count']}")


Total number of HTML tags: 8

Total number of HTML tags with attributes: 1


#4

In [4]:
import requests

def get_public_ip() -> str:
    response = requests.get('https://api.ipify.org')
    public_ip = response.text
    return public_ip

public_ip = get_public_ip()
print(f"The current public IP address is: {public_ip}")

The current public IP address is: 35.202.122.218


#5

In [5]:
%%file comparison.py

def compare_versions(version1: str, version2: str) -> int:
    version1_list = list(map(int, version1.split(".")))
    version2_list = list(map(int, version2.split(".")))
    length = max(len(version1_list), len(version2_list))
    for i in range(length):
        v1 = version1_list[i] if i < len(version1_list) else 0
        v2 = version2_list[i] if i < len(version2_list) else 0
        if v1 > v2:
            return 1
        elif v1 < v2:
            return -1
    return 0

Overwriting comparison.py


#Автотесты для 5

In [6]:
%%file test_version_comparison.py

import pytest
import sys
import os

py_file_location = "/content"
sys.path.append(os.path.abspath(py_file_location))

from comparison import compare_versions

def test_first_is_newer():
  assert compare_versions("1.1", "1.0") == 1

def test_first_is_older():
  assert compare_versions("1.0", "1.1") == -1

def test_equal_versions():
  assert compare_versions("1.0", "1.0") == 0

def test_first_long_is_newer():
  assert compare_versions("1.10.10.5", "1.9.10.5") == 1

def test_second_long_is_newer():
  assert compare_versions("3.14.15.5", "3.14.17.5") == -1

Overwriting test_version_comparison.py


In [7]:
!python -m pytest test_version_comparison.py

============================= test session starts ==============================
platform linux -- Python 3.8.10, pytest-3.6.4, py-1.11.0, pluggy-0.7.1
rootdir: /content, inifile:
plugins: typeguard-2.7.1
collected 5 items                                                              

test_version_comparison.py .....                                         [100%]

=========================== 5 passed in 0.03 seconds ===========================
